In [1]:
#Import all the libraries necessary to this research

#Data Preparation and Analysis
import pandas as pd
import numpy as np

#Data Visualization 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


# Association Rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

# Disable Warnings
from warnings import filterwarnings

In [2]:
filterwarnings('ignore') 

# A1. PROPOSAL OF QUESTION
Which set of products are frequently bought together concurrently?






# 






# A2. DEFINED GOAL
The focus of this analysis is to utilize the concept of Market Basket Analysis in order to uncover associations between products, revealing which items tend to be bought together as a cohesive set.



#  






# B1. EXPLANATION OF MARKET BASKET

Market basket analysis (MBA) is a data mining technique widely employed in retail settings to uncover purchase patterns. It utilizes statistical affinity calculations to gain insights into customer behavior and enhance customer service by identifying frequent combinations of products. In essence, MBA aims to determine which products are most commonly purchased together in transactions. These discovered relationships can be leveraged to increase profitability through cross-selling, recommendations, promotions, or strategic placement of items in stores or menus (Smartbridge, 2022).

The implementation of market basket analysis relies on the use of association rule mining, a rule-based machine learning method. Association rules are simple If/Then statements that reveal correlations between different products based on their co-occurrence in a dataset (Lim, 2022). An association rule comprises an antecedent (the "if" part) and a consequent (the "then" part). For example, a rule might state, "If a customer buys bread, there is a 70% likelihood they will also purchase milk." In this case, bread is the antecedent and milk is the consequent.

In this experiment, association rules are discovered by analyzing data to identify persistent if-then patterns. Metrics such as support, lift, and confidence are used to assess the significance of relationships. Support indicates the frequency of the if/then relationship in the database, while confidence measures the reliability of these relationships. Lift represents the ratio of the observed frequency of co-occurrence of items to the expected frequency (Rai, 2019). These metrics aid in identifying the most important and meaningful relationships among the analyzed products.

# 








# B2. TRANSACTION EXAMPLE

In [3]:
# Load the data set
telco_data = pd.read_csv('teleco_market_basket.csv') 
pd.options.display.max_columns = None

# View the head of data set
telco_data.head()   

,Item01,Item02,Item03,Item04,Item05,Item06,Item07,Item08,Item09,Item10,Item11,Item12,Item13,Item14,Item15,Item16,Item17,Item18,Item19,Item20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Logitech M510 Wireless mouse,HP 63 Ink,HP 65 ink,nonda USB C to USB Adapter,10ft iPHone Charger Cable,HP 902XL ink,Creative Pebble 2.0 Speakers,Cleaning Gel Universal Dust Cleaner,Micro Center 32GB Memory card,YUNSONG 3pack 6ft Nylon Lightning Cable,TopMate C5 Laptop Cooler pad,Apple USB-C Charger cable,HyperX Cloud Stinger Headset,TONOR USB Gaming Microphone,Dust-Off Compressed Gas 2 pack,3A USB Type C Cable 3 pack 6FT,HOVAMP iPhone charger,SanDisk Ultra 128GB card,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple Lightning to Digital AV Adapter,TP-Link AC1750 Smart WiFi Router,Apple Pencil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Explore data

telco_data.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item01  7501 non-null   object
 1   Item02  5747 non-null   object
 2   Item03  4389 non-null   object
 3   Item04  3345 non-null   object
 4   Item05  2529 non-null   object
 5   Item06  1864 non-null   object
 6   Item07  1369 non-null   object
 7   Item08  981 non-null    object
 8   Item09  654 non-null    object
 9   Item10  395 non-null    object
 10  Item11  256 non-null    object
 11  Item12  154 non-null    object
 12  Item13  87 non-null     object
 13  Item14  47 non-null     object
 14  Item15  25 non-null     object
 15  Item16  8 non-null      object
 16  Item17  4 non-null      object
 17  Item18  4 non-null      object
 18  Item19  3 non-null      object
 19  Item20  1 non-null      object
dtypes: object(20)
memory usage: 2.3+ MB


In [5]:
# Drop all null records

telco_data = telco_data.dropna(axis = 0, how = 'all')    #(ref F1)

In [6]:
# For each row, create a list of transactions

tr_data = list(telco_data.values)

In [7]:
# Remove nan values from. each transaction

nan = float('nan')
tr_data = [[m for m in n if m == m] for n in tr_data]

In [8]:
# Output a transaction

print(tr_data[11])

['USB Type C to USB-A Charger cable']


# 







# B3. MARKET BASKET ASSUMPTION
Market basket analysis operates on the premise that when two or more products frequently appear together in a basket, it indicates a complementary relationship in purchases, if not in consumption. As a result, the purchase of one product is likely to lead to the purchase of others (Kamakura, 2012).

# 









# C1. TRANSFORMING THE DATASET

In [9]:
# Instantiate Transaction. Encoder Object

te_data = TransactionEncoder().fit(tr_data)   #(ref F2)

In [10]:
# One Hot Enocde all of the transactions

ohe_data = te_data.transform(tr_data)
ohe_data

array([[ True, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [11]:
# New Pandas Dataframe. of all the encoded transactions

enc_df = pd.DataFrame(ohe_data, columns = te_data.columns_)
enc_df.head()

,10ft iPHone Charger Cable,10ft iPHone Charger Cable 2 Pack,3 pack Nylon Braided Lightning Cable,3A USB Type C Cable 3 pack 6FT,5pack Nylon Braided USB C cables,ARRIS SURFboard SB8200 Cable Modem,Anker 2-in-1 USB Card Reader,Anker 4-port USB hub,Anker USB C to HDMI Adapter,Apple Lightning to Digital AV Adapter,Apple Lightning to USB cable,Apple Magic Mouse 2,Apple Pencil,Apple Pencil 2nd Gen,Apple Power Adapter Extension Cable,Apple USB-C Charger cable,AutoFocus 1080p Webcam,BENGOO G90000 headset,Blue Light Blocking Glasses,Blue Light Blocking Glasses 2pack,Brother Genuine High Yield Toner Cartridge,Cat 6 Ethernet Cable 50ft,Cat8 Ethernet Cable,CicTsing MM057 2.4G Wireless Mouse,Cleaning Gel Universal Dust Cleaner,Creative Pebble 2.0 Speakers,DisplayPort ot HDMI adapter,Dust-Off Compressed Gas,Dust-Off Compressed Gas 2 pack,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses,Falcon Dust Off Compressed Gas,HOVAMP Mfi 6pack Lightning Cable,HOVAMP iPhone charger,HP 61 2 pack ink,HP 61 Tri-color ink,HP 61 ink,HP 62XL Tri-Color ink,HP 62XL ink,HP 63 Ink,HP 63 Tri-color ink,HP 63XL Ink,HP 63XL Tri-color ink,HP 64 Tri-Color ink,HP 64 ink,HP 65 ink,HP 902XL ink,HP 952 ink,HP ENVY 5055 printer,HP952XL ink,HooToo USB C Hub,HyperX Cloud Stinger Headset,Jelly Comb 2.4G Slim Wireless mouse,Leader Desk Pad Protector,Logitech M510 Wireless mouse,Logitech MK270 Wireless Keyboard/Mouse,Logitech MK345 Wireless combo,Logitech USB H390 headset,M.2 Screw kit,Mfi-Certified Lightning to USB A Cable,Micro Center 32GB Memory card,Microsot Surface Dock 2,Moread HDMI to VGA Adapter,Mpow HC6 USB Headset,NETGEAR CM500 Cable Modem,NETGEAR Nighthawk WiFi Router,NETGEAR Orbi Home Mesh WiFi System,Nylon Braided Lightning to USB cable,PS4 Headset,Premium Nylon USB Cable,RUNMUS Gaming Headset,SAMSUNG 128GB card,SAMSUNG 256 GB card,SAMSUNG EVO 32GB card,SAMSUNG EVO 64GB card,Sabrent 4-port USB 3.0 hub,SanDisk 128GB Ultra microSDXC card,SanDisk 128GB card,SanDisk 128GB microSDXC card,SanDisk 32GB Ultra SDHC card,SanDisk 32GB card,SanDisk Extreme 128GB card,SanDisk Extreme 256GB card,SanDisk Extreme 32GB 2pack card,SanDisk Extreme Pro 128GB card,SanDisk Extreme Pro 64GB card,SanDisk Ultra 128GB card,SanDisk Ultra 256GB card,SanDisk Ultra 400GB card,SanDisk Ultra 64GB card,Screen Mom Screen Cleaner kit,Stylus Pen for iPad,Syntech USB C to USB Adapter,TONOR USB Gaming Microphone,TP-Link AC1750 Smart WiFi Router,TP-Link AC4000 WiFi router,TopMate C5 Laptop Cooler pad,UNEN Mfi Certified 5-pack Lightning Cable,USB 2.0 Printer cable,USB C to USB Male Adapter,USB Type C Cable,USB Type C to USB-A Charger cable,VIVO Dual LCD Monitor Desk mount,VicTsing Mouse Pad,VicTsing Wireless mouse,Vsco 70 pack stickers,Webcam with Microphone,XPOWER A-2 Air Pump blower,YUNSONG 3pack 6ft Nylon Lightning Cable,hP 65 Tri-color ink,iFixit Pro Tech Toolkit,iPhone 11 case,iPhone 12 Charger cable,iPhone 12 Pro case,iPhone 12 case,iPhone Charger Cable Anker 6ft,iPhone SE case,nonda USB C to USB Adapter,seenda Wireless mouse
0,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False,False,True,True,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [12]:
# Save new ataframe as an Execel Document

enc_df.to_excel('encoded_teleco_market_basket_data.xlsx', index = False, encoding = 'utf-8') 

# 









# C2. CODE EXECUTION


In [13]:
# Frequent Itemsets after applying Apriori Algorithm
 
freq_data = apriori(enc_df, min_support = 0.005, use_colnames = True)  #(ref F3)
freq_data.head()

,support,itemsets
0,0.009065,(10ft iPHone Charger Cable)
1,0.050527,(10ft iPHone Charger Cable 2 Pack)
2,0.005199,(3 pack Nylon Braided Lightning Cable)
3,0.042528,(3A USB Type C Cable 3 pack 6FT)
4,0.019064,(5pack Nylon Braided USB C cables)


In [14]:
# Print Total Frequent Item Sets

print("Total Frequent Item Sets .:", freq_data.shape[0])

Total Frequent Item Sets .: 725


In [15]:
# Generate Association Rules, with Support Metric

ar_ = association_rules(freq_data, min_threshold = 0.05, metric = "support")  #(ref F4)

In [16]:
# Total Count of Association Rules

print("The total count of association rules is determined to be ", ar_.shape[0],
      " after applying a support metric with a minimum threshold of 1.")

The total count of association rules is determined to be  6  after applying a support metric with a minimum threshold of 1.


# 




# C3. ASSOCIATION RULES TABLE

In [17]:
# Print the association rules table

ar_

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Apple Pencil),(Dust-Off Compressed Gas 2 pack),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815,0.193648
1,(Dust-Off Compressed Gas 2 pack),(Apple Pencil),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158,0.208562
2,(Dust-Off Compressed Gas 2 pack),(HP 61 ink),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256,0.339197
3,(HP 61 ink),(Dust-Off Compressed Gas 2 pack),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357,0.308965
4,(Dust-Off Compressed Gas 2 pack),(VIVO Dual LCD Monitor Desk mount),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
5,(VIVO Dual LCD Monitor Desk mount),(Dust-Off Compressed Gas 2 pack),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437


# 







# C4. TOP THREE RULES

In [18]:
# Association rules, Order by the criteria: Support, Confidence, and Lift Metric

ar_ = ar_.sort_values(by=['support', 'confidence', 'lift'], ascending = False)
ar_

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
5,(VIVO Dual LCD Monitor Desk mount),(Dust-Off Compressed Gas 2 pack),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437
4,(Dust-Off Compressed Gas 2 pack),(VIVO Dual LCD Monitor Desk mount),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
3,(HP 61 ink),(Dust-Off Compressed Gas 2 pack),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357,0.308965
2,(Dust-Off Compressed Gas 2 pack),(HP 61 ink),0.238368,0.163845,0.052660,0.220917,1.348332,0.013604,1.073256,0.339197
0,(Apple Pencil),(Dust-Off Compressed Gas 2 pack),0.179709,0.238368,0.050927,0.283383,1.188845,0.008090,1.062815,0.193648
1,(Dust-Off Compressed Gas 2 pack),(Apple Pencil),0.238368,0.179709,0.050927,0.213647,1.188845,0.008090,1.043158,0.208562


In [19]:
# Top three Rules after sorting data

ar_.head(3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
5,(VIVO Dual LCD Monitor Desk mount),(Dust-Off Compressed Gas 2 pack),0.174110,0.238368,0.059725,0.343032,1.439085,0.018223,1.159314,0.369437
4,(Dust-Off Compressed Gas 2 pack),(VIVO Dual LCD Monitor Desk mount),0.238368,0.174110,0.059725,0.250559,1.439085,0.018223,1.102008,0.400606
3,(HP 61 ink),(Dust-Off Compressed Gas 2 pack),0.163845,0.238368,0.052660,0.321400,1.348332,0.013604,1.122357,0.308965


In [27]:
# Antecedents and Consequents. of the top 3 rules

ant = list(ar_['antecedents'])
con = list(ar_['consequents'])


print('1st Rule')
print("Antecedent :", ([r for r in ant[0]]))
print("Consequent :", ([r for r in con[0]]))
print('\n')

print('2nd Rule')
print("Antecedent :", ([r for r in ant[1]]))
print("Consequent :", ([r for r in con[1]]))
print('\n')

print('3rd Rule')
print("Antecedent :", ([r for r in ant[2]]))
print("Consequent :", ([r for r in con[2]]))

1st Rule
Antecedent : ['VIVO Dual LCD Monitor Desk mount']
Consequent : ['Dust-Off Compressed Gas 2 pack']


2nd Rule
Antecedent : ['Dust-Off Compressed Gas 2 pack']
Consequent : ['VIVO Dual LCD Monitor Desk mount']


3rd Rule
Antecedent : ['HP 61 ink']
Consequent : ['Dust-Off Compressed Gas 2 pack']


In [28]:
# Top 3 association. rules

print('1st Rule')
print(" If", ([r for r in ant[0]]), "then", ([r for r in con[0]]))
print('\n')

print('2nd Rule')
print(" If", ([r for r in ant[1]]), "then", ([r for r in con[1]]))
print('\n')

print('3rd Rule')
print(" If", ([r for r in ant[2]]), "then", ([r for r in con[2]]))

1st Rule
 If ['VIVO Dual LCD Monitor Desk mount'] then ['Dust-Off Compressed Gas 2 pack']


2nd Rule
 If ['Dust-Off Compressed Gas 2 pack'] then ['VIVO Dual LCD Monitor Desk mount']


3rd Rule
 If ['HP 61 ink'] then ['Dust-Off Compressed Gas 2 pack']


# 







# D1. SIGNIFICANCE OF SUPPORT, LIFT, AND CONFIDENCE SUMMARY

Based on the association rules table, the top rules exhibit relatively low support, with values ranging from 0.052 to 0.060. This indicates that the frequency of occurrence for both the antecedent and consequent itemsets is low withinin the transactions.

Furthermore, the confidence values for the top rules are in the 0.21 to 0.34 range, meaning that when a customer purchases the antecedent item, they only purchase the consequent at most 34.3% of the time. This shows a weak relationship between the antecedent and consequent items.

However, the lift values are higher at 1.34 to 1.44, indicating the presence of a positive association between the antecedent and consequent items. The lift exceeds 1, meaning the relationship is stronger than expected by chance. As mentioned earlier, lift represents the ratio of the observed frequency of co-occurrence to the expected frequency. A lift ratio greater than 1.0 indicates a significant relationship between the antecedent and consequent, surpassing what would be expected if the two sets were independent. Therefore, the larger the lift ratio, the stronger the association between the items.

 



# 






# D2. PRACTICAL SIGNIFICANCE OF FINDINGS

The top three association rules table reveals a significant observation: the first and second rule exhibit identical values for lift, support, and leverage ratios. This similarity arises from the fact that they feature different combinations of antecedent and consequent values. Specifically, the first rule involves VIVO Dual LCD Monitor Desk mount as the antecedent with a support of 0.174110, and Dust-Off Compressed Gas 2 pack as the consequent with a support of 0.238368. On the other hand, the second rule showcases alternative values for both the antecedent and consequent, accompanied by distinct support values.

Another noteworthy finding is the presence of a strong relationship among all three rules, as indicated by their lift ratios surpassing 1. This implies a robust association among the items involved in each rule, suggesting a significant connection between them.






# 





# D3. COURSE OF ACTION
To increase sales, retailers should prominently display and recommend Dust-Off Compressed Gas 2 packs when a customer purchases VIVO Dual LCD Monitor Desk mount, and vice versa. This can be done by prominently displaying related products on product details or checkout pages. Additionally, the companies should ensure that the stocking levels of both products are synchronized, so that there is never a shortage when it comes to the purchase of these complementary items. By maintaining a consistent availability of both products, sales can be maximized.

Furthermore, it is crucial to apply similar concepts to other antecedent-consequent pairs, particularly those with high lift ratios above 1, indicating a strong relationship. By identifying such associations and promoting the corresponding products together, companies can further enhance cross-selling opportunities and drive sales.

Overall, the recommended strategy involves leveraging the identified product relationships to optimize sales. By providing targeted recommendations and synchronizing stock levels, telecommunication companies can effectively capitalize on the purchasing patterns observed and increase customer satisfaction.

# 









# E. PANOPTO RECORDING


# 












# F. SOURCES FOR THIRD-PARTY CODE

F1: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html

F2: https://python.hotexamples.com/examples/mlxtend.preprocessing/TransactionEncoder/-/python-transactionencoder-class-examples.html

F3: https://www.section.io/engineering-education/apriori-algorithm-in-python/

F4: https://medium.com/@mervetorkan/association-rules-with-python-9158974e761a









# 












# G. SOURCES

1. Smartbridge (2022, March 15). Market Basket Analysis 101: Anticipating Customer Behavior. Retrieved July 18, 2023, at https://smartbridge.com/market-basket-analysis-101/

2. Lim Y (2022, April 8). Data Mining: Market Basket Analysis with Apriori Algorithm. Retrieved July 18, 2023, at https://towardsdatascience.com/data-mining-market-basket-analysis-with-apriori-algorithm-970ff256a92c

3. Rai A (2019, June 4). An Overview of Association Rule Mining & its Applications. Retrieved July 18, 2023, at https://www.upgrad.com/blog/association-rule-mining-an-overview-and-its-applications/

4. Kamakura W A (2012, May 222). Sequential market basket analysis. Retrieved July 18, 2023, at http://wak2.web.rice.edu/bio/My%20Reprints/Sequential%20Market%20Basket%20Analysist.pdf